In [13]:
import numpy as np  # Импорт библиотеки NumPy для работы с массивами и математическими функциями.
import pandas as pd  # Импорт библиотеки Pandas для работы с данными в форме таблицы (DataFrame).
from scipy.stats import f  # Импорт функции f из библиотеки SciPy для работы с распределением F.
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  # Импорт класса LinearDiscriminantAnalysis из библиотеки scikit-learn для выполнения анализа дискриминантной функции.
from scipy.spatial.distance import mahalanobis  # Импорт функции mahalanobis из библиотеки SciPy для вычисления расстояния Махаланобиса.

In [40]:
FEATURES = ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9"]
# Определение списка FEATURES, содержащего названия признаков, которые будут использоваться в анализе.

TRAIN_SAMPLES = {
    1: [3, 5, 7, 9, 13,14,16,17,23,24,26],
    2: [32, 82],
    3: [20,25,28,39,45,68],
    4: [15,19,43,47,60],
    5: [8,21,22,41,42,51]
}
# Определение словаря TRAIN_SAMPLES, где ключи - номера классов, а значения - индексы обучающих примеров для каждого класса.

data = pd.read_excel(r'norm_data.xlsx', usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
# Загрузка данных из Excel-файла в объект DataFrame библиотеки Pandas, используя только указанные столбцы.

data = data.loc[range(0, 85)]
# Выборка первых 85 строк из данных, чтобы оставить только нужное количество образцов.

print("Данные")
print(data.head())
# Вывод первых нескольких строк данных для визуального ознакомления с ними.

data_to_excel = data[FEATURES]
# Выборка только тех столбцов данных, которые соответствуют признакам из списка FEATURES.

Данные
                                     Регион        X1        X2        X3  \
0                            Алтайский край  0.423721  0.439726 -0.646954   
1                          Амурская область  1.932733  1.687317  0.722511   
2  Архангельская область без автономного ок -0.667681  0.589500  1.640880   
3                      Астраханская область  0.097353 -0.503612 -0.666108   
4                      Белгородская область -0.923862 -0.523216 -0.730266   

         X4        X5        X6        X7        X8        X9  
0  1.676521  0.920458 -0.832257  0.922800 -0.626353  0.089137  
1  0.050502  1.192731  0.517564  0.966786  0.206284 -0.368074  
2 -0.664946  0.235605  0.155037  0.652055  0.095233 -0.498706  
3 -0.274702 -0.707612 -0.359749 -0.375588 -0.576026  0.565010  
4 -0.534865  1.825977 -0.222914 -0.842681 -0.039178  0.723634  


In [20]:
def get_train_data(data, features, train_samples=None):
    # Определение функции get_train_data, которая принимает три параметра: data (таблица с данными),
    # features (список признаков) и train_samples (словарь с обучающими выборками для каждого класса).

    train_data = pd.DataFrame()
    # Создание нового пустого DataFrame под название train_data для хранения обучающих данных.

    for cls, samples in train_samples.items():
        # Итерация по элементам словаря train_samples, где cls - номер класса, samples - список индексов строк.

        train_samps = data[features].loc[samples]
        # Выборка строк из исходных данных (data) по указанным признакам (features) и номерам строк (samples).

        train_samps["Class"] = cls
        # Добавление новой колонки "Class" с номером класса к выбранным обучающим данным.

        train_data = pd.concat([train_data, train_samps])
        # Объединение текущих обучающих данных с предыдущими в DataFrame train_data.

    train_data = train_data.astype({"Class": 'int32'})
    # Приведение типа данных в колонке "Class" к целочисленному (int32).

    return train_data
    # Возвращение итоговой обучающей выборки.

train_data = get_train_data(data, FEATURES, TRAIN_SAMPLES)
# Вызов функции get_train_data с передачей исходных данных, признаков и обучающих выборок.

print("Обучающая выборка")
print(train_data)
# Вывод на экран обучающей выборки.

data_to_excel["Train sample"] = train_data.Class
# Добавление новой колонки "Train sample" с номерами классов в DataFrame data_to_excel.


Обучающая выборка
          X1        X2        X3        X4        X5        X6        X7  \
3   0.097353 -0.503612 -0.666108 -0.274702 -0.707612 -0.359749 -0.375588   
5  -1.587125  0.754957 -0.260296  1.481399  0.046729 -0.403475 -0.111933   
7  -0.204449 -0.350702 -0.881749 -0.144620 -0.454377 -0.492329 -0.288642   
9   0.507945 -0.411082 -0.495336  0.375706  1.193793 -0.386557 -0.724502   
13  0.237726  0.279759  0.376950 -0.144620 -0.058421 -0.388520  0.058677   
14  0.174558  1.064698 -0.028162  0.310665 -0.657520 -0.653256  0.649385   
16 -0.383425 -1.269731  0.121041  1.286276 -0.356776 -0.291237 -0.665712   
17  0.272819  0.275054  0.278703  0.440747 -2.017089 -0.267983 -0.598811   
23  0.160521 -0.474598 -0.650875  0.050502 -0.625166 -0.420316 -1.076687   
24 -0.004418 -0.052723 -0.157175  0.570828 -0.301720 -0.525390  0.257490   
26  0.455305  0.339355 -0.559515 -1.185272 -0.179424 -0.324500 -0.280374   
32 -1.643275  0.306420  1.614578 -0.404783  2.632094  4.957725  1.7300

In [21]:
def scatter_matrix(samples):
    # Функция для вычисления матрицы рассеивания.
    if isinstance(samples, pd.Series):
        # Проверка, является ли samples объектом pd.Series (одномерным массивом).
        samples = samples.to_frame()
        # Преобразование pd.Series в pd.DataFrame, если необходимо.

    d = samples - samples.mean()
    # Вычисление отклонений значений признаков от их средних значений.

    res = np.zeros((d.shape[1], d.shape[1]))
    # Создание матрицы нулей размерности (число признаков) x (число признаков).

    for _, row in d.iterrows():
        # Итерация по строкам DataFrame d.
        col = row.to_frame()
        # Преобразование строки в pd.DataFrame.
        res += col @ col.T
        # Накапливание в res произведения строки на транспонированную версию этой строки.
    return res
    # Возвращение рассеивающей матрицы.

def classes_scatter_matrix(samples, labels):
    # Функция для вычисления классовой матрицы рассеивания.

    A = np.zeros((samples.shape[1], samples.shape[1]))
    # Создание матрицы нулей размерности (число признаков) x (число признаков).

    for cls in labels.unique():
        # Итерация по уникальным классам в labels.

        A += scatter_matrix(samples[labels == cls])
        # Накапливание в A матриц рассеивания для каждого класса.

    return A
    # Возвращение классовой матрицы рассеивания.

cov = pd.DataFrame(
    classes_scatter_matrix(train_data[FEATURES], train_data.Class) / (train_data.shape[0] - train_data.Class.unique().size),
    index=FEATURES,
    columns=FEATURES
)
# Вычисление ковариационной матрицы как отношение классовой матрицы рассеивания к числу степеней свободы.

print("Ковариационная матрица")
print(cov)
np.cov(train_data)
# Вывод ковариационной матрицы.

Ковариационная матрица
          X1        X2        X3        X4        X5        X6        X7  \
X1  0.286377 -0.014947 -0.017641 -0.211600  0.040500  0.003645  0.006755   
X2 -0.014947  0.483615  0.080012  0.031419 -0.058171  0.000173  0.135692   
X3 -0.017641  0.080012  0.224068  0.079662  0.000880  0.029659  0.117713   
X4 -0.211600  0.031419  0.079662  0.404463 -0.062309  0.001125 -0.040770   
X5  0.040500 -0.058171  0.000880 -0.062309  0.563284 -0.001853 -0.006380   
X6  0.003645  0.000173  0.029659  0.001125 -0.001853  0.024645  0.031719   
X7  0.006755  0.135692  0.117713 -0.040770 -0.006380  0.031719  0.348916   
X8 -0.030367  0.052086  0.044164  0.057350 -0.038022  0.005217 -0.028636   
X9  0.112450  0.111576  0.016095 -0.118128  0.161655 -0.005303  0.019066   

          X8        X9  
X1 -0.030367  0.112450  
X2  0.052086  0.111576  
X3  0.044164  0.016095  
X4  0.057350 -0.118128  
X5 -0.038022  0.161655  
X6  0.005217 -0.005303  
X7 -0.028636  0.019066  
X8  0.107623  0.

array([[ 3.20014858e-01,  4.40822716e-02,  2.52013837e-01,
         2.54152982e-01,  1.54168382e-01,  1.80405919e-01,
         1.76281592e-01,  2.10096365e-01,  1.90735362e-01,
         1.55722643e-01,  2.95180975e-01, -4.88497309e-01,
        -4.27122447e-01,  2.99068235e-01,  3.07499190e-01,
         2.94468744e-01,  3.47886371e-01,  3.15293663e-01,
         2.88638189e-01,  6.92858064e-01,  8.09513994e-01,
         6.84139063e-01,  7.67057054e-01,  6.36305586e-01,
         5.83993370e-01,  6.88506619e-01,  6.00688167e-01,
         6.60150561e-01,  5.97446652e-01,  4.24190483e-01],
       [ 4.40822716e-02,  7.47821794e-01,  1.66693754e-01,
         3.62090168e-02,  7.87438821e-02,  2.30312868e-01,
         3.21631270e-01,  1.84657403e-01,  1.14864499e-01,
         2.24055855e-01, -1.37866036e-01,  1.47016463e-01,
         5.17045019e-01,  3.93748605e-01,  1.36879761e-01,
         4.51081108e-01,  2.22180137e-01,  3.50028638e-01,
         4.31020709e-01,  7.31409238e-01,  2.52388807e-

In [22]:
lda = LinearDiscriminantAnalysis().fit(train_data[FEATURES], train_data.Class)
# Создание и обучение модели линейного дискриминантного анализа (LDA) на обучающей выборке. 
# Результаты обучения (модель) присваиваются переменной lda.

means = pd.DataFrame(lda.means_, index=lda.classes_, columns=FEATURES)
# Извлечение средних значений признаков для каждого класса из обученной модели LDA.
# Создание DataFrame means, где строки - классы, столбцы - признаки, и значения - средние значения.

print("Средние значения")  # Вывод заголовка для вывода средних значений.
print(means)
# Вывод на экран средних значений, представленных в виде DataFrame.


Средние значения
         X1        X2        X3        X4        X5        X6        X7  \
1 -0.024836 -0.031693 -0.265684  0.251537 -0.374326 -0.410301 -0.286972   
2 -1.867872  1.196044  1.592584  0.180584  1.098622  4.880008  2.028344   
3  0.148823  0.953478 -0.212692 -0.545705  1.268362 -0.304821  0.203389   
4 -0.897893 -1.601272 -1.505144 -0.365759 -0.417260 -0.473374 -1.216846   
5  0.873500  0.388364  1.064465  0.093863 -0.208609 -0.182360  0.544842   

         X8        X9  
1 -0.219452  0.166328  
2  3.480449 -2.084949  
3 -0.416135 -0.214115  
4 -0.423893 -0.006038  
5 -0.218117 -0.377405  


In [23]:
def find_mahl_sqr_dist(centers, samples, covr):
    # Функция для вычисления квадрата махаланобисовского расстояния между центральными точками (centers) и образцами (samples).
    # Принимает центральные точки (centers), образцы (samples) и матрицу ковариации (covr).

    res = pd.DataFrame(index=samples.index, columns=centers.index)
    # Создание нового DataFrame с номерами образцов в индексах и номерами центральных точек в столбцах.

    for i in centers.index:
        # Внешний цикл: итерация по номерам центральных точек.

        for j in samples.index:
            # Внутренний цикл: итерация по номерам образцов.

            res[i][j] = mahalanobis(centers.loc[i], samples.loc[j], np.linalg.inv(covr)) ** 2
            # Вычисление квадрата махаланобисовского расстояния между центральной точкой и образцом.
            # mahalanobis - функция для вычисления расстояния Махаланобиса.
            # np.linalg.inv(covr) - обратная матрица к матрице ковариации.

    return res
    # Возвращение DataFrame с квадратами махаланобисовских расстояний.

cen_dis = find_mahl_sqr_dist(means, means, cov)
# Вызов функции средних значений и матрицы ковариации для вычисления расстояний Махаланобиса.

print("Расстояние Махаланобиса (обучающая выборка)")
print(cen_dis)
# Вывод на экран расстояний Махаланобиса для обучающей выборки.

Расстояние Махаланобиса (обучающая выборка)
             1            2            3            4            5
1          0.0  1456.967956    21.317349    18.927217    19.190156
2  1456.967956          0.0  1374.939968  1436.444491  1438.018922
3    21.317349  1374.939968          0.0    50.503674    20.051212
4    18.927217  1436.444491    50.503674          0.0    67.304501
5    19.190156  1438.018922    20.051212    67.304501          0.0


In [33]:
def get_def_coef(lda, features):
    # Функция для получения дискриминантных коэффициентов.
    # Принимает обученную модель LDA (lda) и список признаков (features).

    return pd.DataFrame(
        np.vstack([lda.intercept_, lda.coef_.T]),
        # Создание DataFrame из массива, содержащего результаты дискриминантного анализа:
        # np.vstack объединяет массивы, lda.intercept_ - свободный член, lda.coef_.T - транспонированные коэффициенты при признаках.
        # Интерпретация: строки - "Const" и признаки, столбцы - номера классов из модели LDA.

        index=["Const"] + features,
        # Установка индекса DataFrame: первая строка - "Const", затем идут названия признаков.

        columns=lda.classes_
        # Установка столбцов DataFrame: номера классов из обученной модели LDA.
    )

df_coef = get_def_coef(lda, FEATURES)
# Получение дискриминантных коэффициентов для обученной модели LDA и признаков из списка FEATURES.

print("Функции Фишера")
print(df_coef)
# Вывод на экран дискриминантных коэффициентов (функций Фишера).

print("Pi: ", lda.priors_)
# Вывод на экран априорных вероятностей классов, полученных из обученной модели LDA.

Функции Фишера
               1           2          3          4          5
Const  -5.110794 -626.303789 -10.235374 -14.888787 -11.517630
X1      0.633234   -9.826144   1.393362  -6.769874   6.362652
X2     -1.639584    8.539650   3.697875  -4.857890   0.509720
X3      1.022950  -26.332837   1.585004  -3.078547   7.882656
X4      2.201717   -7.270038  -1.883410  -1.409823   1.445127
X5     -2.011239    8.055049   3.633191  -3.294874   0.114792
X6    -18.254729  232.793915 -14.894718 -12.544178 -18.782768
X7      1.032431   -7.184682  -0.001247  -0.023243   0.522719
X8     -2.107289   31.007344  -1.972504  -2.254886  -2.620842
X9      3.637884   -9.202921  -5.686363   7.161101  -3.883036
Pi:  [0.36666667 0.06666667 0.2        0.16666667 0.2       ]


In [34]:
def LDA_predict(lda, x):
    # Функция для предсказания классов объектов с использованием обученной модели LDA.
    # Принимает модель LDA (lda) и значения признаков объектов (x).

    return pd.DataFrame(
        lda.predict(x),
        # Применение метода predict модели LDA к значениям признаков объектов.
        # Полученные предсказанные классы оборачиваются в DataFrame.

        columns=["Class"],
        # Установка названия столбца в DataFrame: "Class".

        index=x.index
        # Установка индекса DataFrame: индексы объектов из исходной таблицы x.
    )

lda_predict = LDA_predict(lda, data[FEATURES])
# Применение функции предсказания для всех объектов в исходной таблице признаков data.

print("Распределение по классам")
print(lda_predict)
# Вывод на экран предсказанных классов для каждого объекта.

data_to_excel["Result Lda"] = lda_predict
# Добавление предсказанных классов в исходную таблицу data_to_excel.

Распределение по классам
    Class
0       1
1       5
2       3
3       1
4       4
..    ...
80      4
81      1
82      2
83      2
84      5

[85 rows x 1 columns]


In [35]:
samp_dist = find_mahl_sqr_dist(means, data[FEATURES], cov)
# Вычисление квадрата махаланобисовского расстояния для каждого образца в исходных данных.
# Используются средние значения (means) и матрица ковариации (cov), вычисленные на основе обучающей выборки.

print("Расстояние Махланобиса")
print(samp_dist)
# Вывод на экран квадратов махаланобисовских расстояний для каждого образца в исходных данных.

Расстояние Махланобиса
              1            2            3            4            5
0     47.016203  1740.547747    64.080468    93.905716    60.130163
1     91.253598  1085.688844    57.077136   156.847901     54.75496
2     49.009156  1248.210435    33.811597    73.307046    34.632286
3      5.674584  1456.655457    38.764924    11.753016    37.201357
4     26.455455  1289.164407    31.580963    17.878048    62.087731
..          ...          ...          ...          ...          ...
80    97.658199  1425.245033   159.621534    37.354138   187.773043
81    18.708445  1524.267897    22.050963    43.724295    27.703392
82  1428.623815     8.639245  1362.326978  1411.593459  1418.180852
83   800.277031   187.298516   746.356562    767.35641    792.05898
84    23.274012  1405.160253    26.905081     49.06295    19.655209

[85 rows x 5 columns]


In [36]:
def LDA_predict_probab(lda, x):
    # Функция для предсказания апостериорных вероятностей классификации объектов с использованием обученной модели LDA.
    # Принимает модель LDA (lda) и значения признаков объектов (x).

    return pd.DataFrame(
        lda.predict_proba(x),
        # функция возвращает результат в виде объекта DataFrame 
        # Применение метода predict_proba модели LDA к значениям признаков объектов.
        # Полученные апостериорные вероятности классификации оборачиваются в DataFrame.

        columns=lda.classes_,
        # Установка названий столбцов в DataFrame: номера классов из обученной модели LDA.

        index=x.index
        # Установка индекса DataFrame: индексы объектов из исходной таблицы x.
    )

lda_post_prob = LDA_predict_probab(lda, data[FEATURES])
# Применение функции предсказания апостериорных вероятностей для всех объектов в исходной таблице признаков data.

print("Probabilities")
print(lda_post_prob)
# Вывод на экран апостериорных вероятностей классификации для каждого объекта.

Probabilities
                1              2              3              4              5
0    9.991187e-01   0.000000e+00   1.073785e-04   2.987211e-11   7.739542e-04
1    1.657105e-08  3.468306e-225   2.384696e-01   4.298387e-23   7.615304e-01
2    5.519392e-04  3.965331e-265   6.008386e-01   1.328155e-09   3.986095e-01
3    9.787031e-01  1.492089e-316   3.482840e-08   2.129676e-02   7.611280e-08
4    2.926916e-02  3.406298e-277   1.230776e-03   9.695001e-01   2.922257e-10
..            ...            ...            ...            ...            ...
80   1.768325e-13  1.680279e-302   3.381844e-27   1.000000e+00   2.606940e-33
81   9.020163e-01   0.000000e+00   9.250266e-02   1.515894e-06   5.479550e-03
82  2.481025e-308   1.000000e+00  3.369496e-294  5.627340e-305  2.506400e-306
83  4.302979e-133   1.000000e+00  1.200076e-121  2.754030e-126  1.429075e-131
84   2.262571e-01  3.479004e-302   2.008556e-02   2.583346e-07   7.536571e-01

[85 rows x 5 columns]


In [37]:
def wilks_lambda(samples, labels):
    # Функция для вычисления критерия Вилкса.
    # Принимает samples - значения признаков в обучающей выборке и labels - номера классов.
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    # Проверка, если samples - это pd.Series (одномерный массив), преобразует в pd.DataFrame.
    # Вычисление определителей матриц рассеивания.
    dT = np.linalg.det(scatter_matrix(samples))
    dE = np.linalg.det(classes_scatter_matrix(samples, labels))
    return dE / dT
    # Возвращение отношения определителей.
def f_p_value(lmbd, n_obj, n_sign, n_cls):
    # Функция для вычисления F-статистики и p-значения.
    # Принимает lmbd - значение лямбды, n_obj - число объектов в обучающей выборке,
    # n_sign - число признаков в модели, n_cls - число классов в обучающей выборке.
    num = (1 - lmbd) * (n_obj - n_cls - n_sign)
    den = lmbd * (n_cls - 1)
    f_value = num / den
    p = f.sf(f_value, n_cls - 1, n_obj - n_cls - n_sign)
    return f_value, p
    # Возвращение значения F-статистики и p-значения.
def forward(samples, labels, f_in=1e-4):
    # Функция для выполнения пошагового вперёд метода.
    # Принимает samples - значения признаков в обучающей выборке, labels - номера классов, f_in - точность.
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to enter", "P value"]
    # Создание списка названий колонок для таблицы результатов.
    n_cls = labels.unique().size
    n_obj = samples.shape[0]
    # Получение числа уникальных классов и числа объектов в обучающей выборке.
    out = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}
    into = {0: pd.DataFrame(columns=st_columns, dtype=float)}
    # Создание словарей для хранения результатов внутри и вне модели.
    step = 0
    # Инициализация переменной для отслеживания шага.
    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # Расчёт характеристик элементов вне модели.
        for el in out[step].index:
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)
            partial_lmbd = lmbda / model_lmbd
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # Расчёт характеристик элементов в модели.
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size - 1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if out[step].index.size == 0 or out[step]["F to enter"].max() < f_in:
            break
        # Добавление нового элемента.
        el_to_enter = out[step]["F to enter"].idxmax()
        into[step + 1] = into[step].append(out[step].loc[el_to_enter])
        out[step + 1] = out[step].drop(index=el_to_enter)
        step += 1
    return into, out
    # Возвращение результатов внутри и вне модели.
into, out = forward(train_data[FEATURES], train_data.Class)
print("Forward stepwise")
for i, tab in into.items():
    print("Step: ", i)
    print(tab, end="\n\n")

C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\810604693.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step + 1] = into[step].append(out[step].loc[el_to_enter])
C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\810604693.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step + 1] = into[step].append(out[step].loc[el_to_enter])
C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\810604693.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  into[step + 1] = into[step].append(out[step].loc[el_to_enter])
C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\810604693.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Forward stepwise
Step:  0
Empty DataFrame
Columns: [Wilk's lmbd, Partial lmbd, F to enter, P value]
Index: []

Step:  1
    Wilk's lmbd  Partial lmbd  F to enter       P value
X6          1.0      0.011854  520.977832  1.119597e-23

Step:  2
    Wilk's lmbd  Partial lmbd  F to enter       P value
X6     0.188650      0.013581  435.795484  5.053872e-22
X3     0.011854      0.216125   21.761740  1.080835e-07

Step:  3
    Wilk's lmbd  Partial lmbd  F to enter       P value
X6     0.100445      0.013425  422.541343  3.653346e-21
X3     0.004696      0.287150   14.274374  5.394299e-06
X2     0.002562      0.526345    5.174387  4.017661e-03

Step:  4
    Wilk's lmbd  Partial lmbd  F to enter       P value
X6     0.056267      0.014331  378.284741  6.201019e-20
X3     0.002730      0.295374   13.120477  1.306601e-05
X2     0.001378      0.585318    3.896594  1.536407e-02
X5     0.001349      0.597962    3.697910  1.894969e-02

Step:  5
    Wilk's lmbd  Partial lmbd  F to enter       P value


In [29]:
forw_stepwise = into[len(into) - 5].index.tolist()
# Выбираем признаки, которые были включены в модель на пятом предыдущем шаге (назад) метода пошагового вперёдного отбора.

forw_stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[forw_stepwise], train_data.Class)
# Обучаем модель линейного дискриминантного анализа (LDA) на выбранных признаках.

forw_stepwise_coef = get_def_coef(forw_stepwise_lda, forw_stepwise)
# Получаем коэффициенты функции Фишера для выбранных признаков.

print("Функции Фишера")
print(forw_stepwise_coef)
# Выводим коэффициенты функции Фишера.

print("Pi: ", forw_stepwise_lda.priors_)
# Выводим априорные вероятности классов в модели LDA.

forw_stepwise_pred = LDA_predict(forw_stepwise_lda, data[forw_stepwise])
# Получаем прогнозы модели LDA для выбранных признаков.

print("Распределение")
print(forw_stepwise_pred.head())
# Выводим распределение классов для первых нескольких объектов.

data_to_excel["Result forward"] = forw_stepwise_pred
# Записываем результаты прогнозирования в таблицу data_to_excel под именем "Result forward".


Функции Фишера
               1           2          3          4          5
Const  -4.670198 -556.602014  -8.317919 -10.993427  -7.126084
X6    -18.404124  227.020727 -13.233630 -14.510484 -16.606982
X3      1.860905  -25.296117   0.280795  -3.280556   7.473381
X2     -1.304591    9.180909   3.091852  -4.114517  -0.331640
X5     -1.730438    6.057281   3.490146  -2.478606  -0.271265
X9      2.538620   -8.751922  -3.840838   3.941506  -1.180580
Pi:  [0.36666667 0.06666667 0.2        0.16666667 0.2       ]
Распределение
   Class
0      1
1      3
2      5
3      1
4      1


In [30]:
def backward(samples, labels, f_r=10.00):
    # Определение списка названий столбцов для выходных таблиц
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to remove", "P value"]
    # Получение количества уникальных классов в метках
    n_cls = labels.unique().size
    # Получение количества объектов в выборке
    n_obj = samples.shape[0]
    
    # Инициализация словарей для хранения переменных внутри и вне модели
    into = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}
    out = {0: pd.DataFrame(columns=st_columns, dtype=float)}
    # Инициализация счетчика шагов
    step = 0

    # Запуск цикла, который выполняется до достижения условия выхода
    while True:
        # Расчет значения лямбда Уилкса для текущей модели
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        
        # Расчет характеристик для элементов вне модели
        for el in out[step].index:
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)
            partial_lmbd = lmbda / model_lmbd
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        
        # Расчет характеристик для элементов внутри модели
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size - 1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        # Проверка условия выхода
        if into[step].index.size == 0 or into[step]["F to remove"].min() > f_r:
            break
        
        # Удаление элемента с минимальным значением F
        el_to_remove = into[step]["F to remove"].idxmin()
        out[step + 1] = out[step].append(into[step].loc[el_to_remove])
        into[step + 1] = into[step].drop(index=el_to_remove)

        # Инкремент счетчика шагов
        step += 1
    
    # Возвращение окончательных словарей, содержащих переменные внутри и вне модели
    return into, out


# Вызов функции backward с указанными аргументами
into, out = backward(train_data[FEATURES], train_data.Class)

# Вывод результатов для каждого шага
print("Backward stepwise")
for i, tab in into.items():
    print("Step: ", i)
    print(tab, end="\n\n")


C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\2300776718.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step + 1] = out[step].append(into[step].loc[el_to_remove])
C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\2300776718.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step + 1] = out[step].append(into[step].loc[el_to_remove])
C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\2300776718.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out[step + 1] = out[step].append(into[step].loc[el_to_remove])
C:\Users\Максим\AppData\Local\Temp\ipykernel_23920\2300776718.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

Backward stepwise
Step:  0
    Wilk's lmbd  Partial lmbd  F to remove   P value
X1     0.000325      0.621836     2.584594  0.074299
X2     0.000366      0.551760     3.452622  0.030692
X3     0.000311      0.648247     2.306142  0.100174
X4     0.000269      0.751573     1.404807  0.274627
X5     0.000388      0.520461     3.915834  0.019716
X6     0.001150      0.175546    19.960112  0.000003
X7     0.000207      0.974709     0.110278  0.977246
X8     0.000216      0.933882     0.300898  0.873300
X9     0.000390      0.517358     3.964808  0.018837

Step:  1
    Wilk's lmbd  Partial lmbd  F to remove       P value
X1     0.000332      0.624449     2.706355  6.323275e-02
X2     0.000422      0.490985     4.665252  9.254125e-03
X3     0.000370      0.559304     3.545713  2.659877e-02
X4     0.000270      0.765619     1.377599  2.810567e-01
X5     0.000400      0.517261     4.199676  1.416787e-02
X6     0.001328      0.155930    24.359039  4.684939e-07
X8     0.000227      0.912108     

In [38]:
# Выбор индексов переменных, оставшихся после шага обратного пошагового отбора
back_stepwise = into[len(into) - 4].index.tolist()
print(back_stepwise)

# Обучение модели LDA на выбранных переменных
back_stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[back_stepwise], train_data.Class)

# Получение коэффициентов дискриминантных функций
back_stepwise_coef = get_def_coef(back_stepwise_lda, back_stepwise)
print("Функции Фишера")
print(back_stepwise_coef)

# Вывод априорных вероятностей классов
print("Pi: ", back_stepwise_lda.priors_)

# Предсказание классов на основе обученной модели LDA и выбранных переменных
back_stepwise_pred = LDA_predict(back_stepwise_lda, data[back_stepwise])

# Вывод распределения классов для первых нескольких записей
print("Распределение")
print(back_stepwise_pred.head())

# Добавление предсказанных результатов в DataFrame "data_to_excel"
data_to_excel["Result backward"] = back_stepwise_pred


['X6', 'X3', 'X2', 'X5', 'X9', 'X1']
Функции Фишера
               1           2          3          4          5
Const  -4.667252 -567.024229  -9.324429 -14.163523 -11.167041
X6    -18.253605  231.084065 -14.381265 -12.227815 -18.991969
X3      1.817629  -26.464380   0.610756  -3.936854   8.159097
X2     -1.363639    7.586875   3.542065  -5.010001   0.603983
X5     -1.771368    4.952371   3.802213  -3.099313   0.377265
X9      2.754522   -2.923549  -5.486983   7.215723  -4.601560
X1     -0.369498   -9.974796   2.817246  -5.603562   5.854735
Pi:  [0.36666667 0.06666667 0.2        0.16666667 0.2       ]
Распределение
   Class
0      3
1      3
2      5
3      1
4      4


In [32]:
data_to_excel.to_excel(r'Train sample.xlsx')